In [1]:
import warnings; warnings.filterwarnings('ignore')
import networkx as nx
import scprep
from torch_geometric_signed_directed.data.directed import load_directed_real_data
from GraphRicciCurvature.OllivierRicci import OllivierRicci
import pandas as pd
import numpy as np
import graphtools
from tqdm import tqdm
import scipy
import omnipath as op
import os, pandas as pd, numpy as np
import networkx as nx

import rpy2, rpy2.situation
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
import seaborn as sns
for i in rpy2.situation.iter_info(): # Print Rpy2 info
    print(i)

pandas2ri.activate()
%load_ext rpy2.ipython

utils = importr('utils')
utils.install_packages('sna')
base = importr("base")
sna = importr("sna")

rpy2 version:
3.5.1
Python version:
3.8.5 (default, Sep  5 2020, 10:50:12) 
[GCC 10.2.0]
Looking for R's HOME:
    Environment variable R_HOME: /usr/lib64/R
    Calling `R RHOME`: /usr/lib64/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:
/usr/lib64/R/lib:/usr/lib/jvm/java-14-openjdk/lib/server
R version:
    In the PATH: R version 4.0.2 (2020-06-22) -- "Taking Off Again"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/usr/include/R/']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'tirpc', 'rt', 'dl', 'm', 'icuuc', 'icui18n']
  library_dirs:
  ['/usr/lib64/R/lib']
  extra_compile_args:
  []
  extra_link_args:
  ['-Wl,--export-dynamic', '-fopenmp', '-Wl,-O1,--sort-common,--as-needed,-z,relro,-z,now']


(as ‘lib’ is unspecified)



--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Brussels) [https]
 8: Brazil (PR) [https]
 9: Brazil (SP 1) [https]
10: Brazil (SP 2) [https]
11: Bulgaria [https]
12: Canada (MB) [https]
13: Canada (ON) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Guangzhou) [https]
20: China (Jinan) [https]
21: China (Lanzhou) [https]
22: China (Nanjing) [https]
23: China (Shanghai 2) [https]
24: China (Shenzhen) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: France (Lyon 1) [https]
33: France (Lyon 2) [https]
34: France (Marseille) [ht







	‘/tmp/RtmpS67arK/downloaded_packages’



In [2]:
"""
Code from https://github.com/HazyResearch/hgcn, with addition of mean Gromov hyperbolicity as computed in https://arxiv.org/abs/1810.06546
"""
def hyperbolicity_sample(G, num_samples=50000):
    hyps = []
    for i in tqdm(range(num_samples)):
        node_tuple = np.random.choice(G.nodes(), 4, replace=False)
        s = []
        try:
            d01 = nx.shortest_path_length(G, source=node_tuple[0], target=node_tuple[1], weight=None)
            d23 = nx.shortest_path_length(G, source=node_tuple[2], target=node_tuple[3], weight=None)
            d02 = nx.shortest_path_length(G, source=node_tuple[0], target=node_tuple[2], weight=None)
            d13 = nx.shortest_path_length(G, source=node_tuple[1], target=node_tuple[3], weight=None)
            d03 = nx.shortest_path_length(G, source=node_tuple[0], target=node_tuple[3], weight=None)
            d12 = nx.shortest_path_length(G, source=node_tuple[1], target=node_tuple[2], weight=None)
            s.append(d01 + d23)
            s.append(d02 + d13)
            s.append(d03 + d12)
            s.sort()
            hyps.append((s[-1] - s[-2]) / 2)
        except Exception as e:
            #print (e)
            continue
    return max(hyps), np.mean(hyps)

In [4]:
for db in ['Texas', 'Cornell']:
    edges = load_directed_real_data(root='data', name=db)['edge_index'].numpy().T
    G = nx.from_edgelist([tuple(x) for x in edges], create_using=nx.DiGraph)
    print (db, 'Reciprocity', nx.overall_reciprocity(G))
    print (db, 'Khs', sna.hierarchy(nx.to_numpy_array(G), measure="krackhardt", ))
    
    orc = OllivierRicci(G, alpha=0.5, verbose="ERROR")
    orc.compute_ricci_curvature()
    G_orc = orc.G.copy()

    curv = []
    for n1,n2 in list(G_orc.edges()):
        curv.append(G_orc[n1][n2]['ricciCurvature'])
    print (db, 'OR Curvature', np.mean(curv))
    
    hyp_max, hyp_mean = hyperbolicity_sample(G.to_undirected(), 10000)
    print (db, 'Max Gromov hyperbolicity', hyp_max)
    print (db, 'Mean Gromov hyperbolicity', hyp_mean)

Texas Reciprocity 0.18461538461538463
Texas Khs [0.91089109]
Texas OR Curvature -0.137444136230544


100%|██████████| 10000/10000 [00:00<00:00, 10305.10it/s]

Texas Max Gromov hyperbolicity 1.0
Texas Mean Gromov hyperbolicity 0.03015
Cornell Reciprocity 0.12080536912751678
Cornell Khs [0.95827124]


Cornell OR Curvature -0.301873562619872


100%|██████████| 10000/10000 [00:01<00:00, 8286.24it/s]


Cornell Max Gromov hyperbolicity 1.5
Cornell Mean Gromov hyperbolicity 0.02515


In [ ]:
db = 'omnipath'
interactions = op.interactions.AllInteractions.get(genesymbols=True)
interactions = interactions[(interactions['consensus_direction']) & (interactions['n_references'] != 'None')]
G = nx.from_pandas_edgelist(df=interactions, source='source_genesymbol', target='target_genesymbol', create_using=nx.DiGraph)

print(db, 'Reciprocity', nx.overall_reciprocity(G))
print (db, 'Khs', sna.hierarchy(nx.to_numpy_array(G), measure="krackhardt", ))

orc = OllivierRicci(G, alpha=0.5, verbose='ERROR')
orc.compute_ricci_curvature()
G_orc = orc.G.copy()

curv = []
for n1,n2 in list(G_orc.edges()):
    curv.append(G_orc[n1][n2]['ricciCurvature'])
print (db, 'OR Curvature', np.mean(curv))

hyp_max, hyp_mean = hyperbolicity_sample(G.to_undirected(), 10000)
print (db, 'Max Gromov hyperbolicity', hyp_max)
print (db, 'Mean Gromov hyperbolicity', hyp_mean)

omnipath Reciprocity 0.04446782871399718


In [ ]:
dbs = ['SIGNOR', 'iPTMnet']
types = ['post_translational', 'post_translational']

for i in range(2):
    db = dbs[i]
    type = types[i]
    print (db, type)
    interactions_sub = interactions[interactions['type'] == type]
    df = interactions_sub[[db in x for x in interactions_sub['sources'].str.split(';')]]
    G = nx.from_pandas_edgelist(df=df, source='source_genesymbol', target='target_genesymbol', create_using=nx.DiGraph)
    
    print('Reciprocity', nx.overall_reciprocity(G)
    print ('Khs', sna.hierarchy(nx.to_numpy_array(G), measure="krackhardt", ))
    
    orc = OllivierRicci(G, alpha=0.5, verbose="ERROR")
    orc.compute_ricci_curvature()
    G_orc = orc.G.copy()

    curv = []
    for n1,n2 in list(G_orc.edges()):
        curv.append(G_orc[n1][n2]['ricciCurvature'])
    print (db, 'OR Curvature', np.mean(curv))
    
    hyp_max, hyp_mean = hyperbolicity_sample(G.to_undirected(), 10000)
    print (db, 'Max Gromov hyperbolicity', hyp_max)
    print (db, 'Mean Gromov hyperbolicity', hyp_mean)

## Graph statistics from directed scattering

In [ ]:
for dataset in ['Texas', 'Cornell', 'SIGNOR', 'omnipath', 'iPTMnet']:
    config = np.load(f'results/DS-PM/{dataset}/best_val.npz', allow_pickle=True)['config'][()]
    ds = pd.read_csv(f'results/Directed_Scattering/Directed_Scattering_J{config["J"]}_q{config["q"]}_{dataset}_train_val_embedding.csv', compression='gzip', index_col=0)

    G = graphtools.Graph(ds)
    G_nx = nx.from_numpy_matrix(G.K)
    print('Reciprocity', nx.overall_reciprocity(G_nx.to_undirected()))
    print ('Khs', sna.hierarchy(nx.to_numpy_array(G_nx), measure="krackhardt", ))

    orc = OllivierRicci(G_nx, alpha=0.5, verbose="ERROR")
    orc.compute_ricci_curvature()
    G_orc = orc.G.copy()

    curv = []
    for n1,n2 in list(G_orc.edges()):
        curv.append(G_orc[n1][n2]['ricciCurvature'])
    print (dataset, 'OR Curvature', np.mean(curv))
    hyp_max, hyp_mean = hyperbolicity_sample(G_nx.to_undirected(), 10000)
    print (dataset, 'Max Gromov hyperbolicity', hyp_max)
    print (dataset, 'Mean Gromov hyperbolicity', hyp_mean)